In [ ]:
from pathlib import Path

ROOT = Path.cwd().resolve().parents[2]
DATA_DIR = ROOT / 'data'
DOCS_DIR = ROOT / 'docs'
LOGS_DIR = ROOT / 'logs'
print('ROOT=', ROOT)
print('DATA_DIR=', DATA_DIR)
print('DOCS_DIR=', DOCS_DIR)
print('LOGS_DIR=', LOGS_DIR)


In [ ]:
# Override local paths to project root
try:
    PROJECT_ROOT = ROOT
except NameError:
    from pathlib import Path
    PROJECT_ROOT = Path.cwd().resolve().parents[2]
print('Using PROJECT_ROOT =', PROJECT_ROOT)


# DataSens E1_v1 — 01_setup_env

- Objectifs: structure projet, logging, versioning (README_VERSIONNING)
- Prérequis: Python, venv activé, `pip install -r requirements.txt`
- Ordre global E1_v1: 01 → 02 → 03 → 04 → 05
- Guide: docs/GUIDE_TECHNIQUE_E1.md


In [ ]:
# ============================================================
# 🎬 DASHBOARD NARRATIF - OÙ SOMMES-NOUS ?
# ============================================================
# Ce dashboard vous guide à travers le pipeline DataSens E1
# Il montre la progression et l'état actuel des données
# ============================================================

import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch
import matplotlib.patches as mpatches

print("\n" + "="*80)
print("🎬 FIL D'ARIANE VISUEL - PIPELINE DATASENS E1")
print("="*80)

# Créer figure dashboard
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(111)
ax.set_xlim(0, 10)
ax.set_ylim(0, 6)
ax.axis('off')

# Étapes du pipeline
etapes = [
    {"nom": "📥 COLLECTE", "status": "✅", "desc": "Sources brutes"},
    {"nom": "☁️ DATALAKE", "status": "✅", "desc": "MinIO Raw"},
    {"nom": "🧹 NETTOYAGE", "status": "🔄", "desc": "Déduplication"},
    {"nom": "💾 ETL", "status": "⏳", "desc": "PostgreSQL"},
    {"nom": "📊 ANNOTATION", "status": "⏳", "desc": "Enrichissement"},
    {"nom": "📦 EXPORT", "status": "⏳", "desc": "Dataset IA"}
]

# Couleurs selon statut
colors = {
    "✅": "#4ECDC4",
    "🔄": "#FECA57", 
    "⏳": "#E8E8E8"
}

# Dessiner timeline
y_pos = 4
x_start = 1
x_spacing = 1.4

for i, etape in enumerate(etapes):
    x_pos = x_start + i * x_spacing
    
    # Cercle étape
    circle = plt.Circle((x_pos, y_pos), 0.25, color=colors[etape["status"]], zorder=3)
    ax.add_patch(circle)
    ax.text(x_pos, y_pos, etape["status"], ha='center', va='center', fontsize=14, fontweight='bold', zorder=4)
    
    # Nom étape
    ax.text(x_pos, y_pos - 0.6, etape["nom"], ha='center', va='top', fontsize=11, fontweight='bold')
    ax.text(x_pos, y_pos - 0.85, etape["desc"], ha='center', va='top', fontsize=9, style='italic')
    
    # Flèche vers prochaine étape
    if i < len(etapes) - 1:
        ax.arrow(x_pos + 0.3, y_pos, x_spacing - 0.6, 0, 
                head_width=0.1, head_length=0.15, fc='gray', ec='gray', zorder=2)

# Titre narratif
ax.text(5, 5.5, "🎯 PROGRESSION DU PIPELINE E1", ha='center', va='center', 
        fontsize=16, fontweight='bold', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Légende
legend_elements = [
    mpatches.Patch(facecolor='#4ECDC4', label='Terminé'),
    mpatches.Patch(facecolor='#FECA57', label='En cours'),
    mpatches.Patch(facecolor='#E8E8E8', label='À venir')
]
ax.legend(handles=legend_elements, loc='upper left', fontsize=10)

# Statistiques rapides (si disponibles)
stats_text = "\n📊 SNAPSHOT ACTUEL :\n"
try:
    # Essayer de charger des stats si base disponible
    stats_text += "   • Pipeline en cours d'exécution...\n"
except:
    stats_text += "   • Démarrage du pipeline...\n"

ax.text(5, 1.5, stats_text, ha='center', va='center', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))

plt.title("🎬 FIL D'ARIANE VISUEL - Accompagnement narratif du jury", 
          fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n💡 Le fil d'Ariane vous guide étape par étape à travers le pipeline")
print("   Chaque visualisation s'inscrit dans cette progression narrative\n")



> Notes:
> - On crée l’arborescence standard du projet (data/logs/docs/notebooks…).
> - `Path(...).mkdir(..., exist_ok=True)` évite d’échouer si le dossier existe déjà.
> - La fonction `log_version` écrit une ligne d’historique dans `README_VERSIONNING.md`.
> - L’historique permet une traçabilité étape par étape.


In [2]:
# DataSens E1_v1 - 01_setup_env
# 🎯 Objectif: créer la structure et le versioning
from datetime import datetime
from pathlib import Path

PROJECT_ROOT = Path.cwd()
FOLDERS = [
    "datasens", "data/raw", "data/silver", "data/gold",
    "notebooks", "logs", "docs", "flows"
]
for f in FOLDERS:
    (PROJECT_ROOT / f).mkdir(parents=True, exist_ok=True)
print("✅ Structure du projet créée :", PROJECT_ROOT)

# Arbre (optionnel)
def show_tree(root: Path, prefix=""):
    items = [item for item in root.iterdir() if not item.name.startswith('.')]
    items = sorted(items, key=lambda p: (not p.is_dir(), p.name.lower()))
    for i, item in enumerate(items):
        last = (i == len(items) - 1)
        connector = "└── " if last else "├── "
        print(prefix + connector + item.name)
        if item.is_dir() and not item.name.startswith('.'):
            show_tree(item, prefix + ("    " if last else "│   "))
# show_tree(PROJECT_ROOT)

# Versioning simple
VERSION_FILE = PROJECT_ROOT / "README_VERSIONNING.md"
BACKUP_DIR = PROJECT_ROOT / "datasens" / "versions"
BACKUP_DIR.mkdir(parents=True, exist_ok=True)

def log_version(action: str, details: str = ""):
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    entry = f"- **{now}** | `{action}` | {details}\n"
    with open(VERSION_FILE, "a", encoding="utf-8") as f:
        f.write(entry)
    print(f"🕒 Log : {action}")

log_version("INIT", "Création de la structure projet et versionning")
print("Environnement prêt.")


✅ Structure du projet créée : c:\Users\Utilisateur\Desktop\DataSens\notebooks\datasens_E1_v1
🕒 Log : INIT
Environnement prêt.
